In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pyspark-3.3.1.tar.gz (281.4 MB)
     |████████████████████████████████| 199 kB 4.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=a9cae458934169c1180bd22123540790a3fca7fa8a9ac3b91caf35f2a965c81c
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Load the packages needed for this part
# create spark and sparkcontext objects
from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import pyspark
from pyspark.ml import feature, regression, Pipeline, classification, pipeline, evaluation
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import functions as fn, Row
from pyspark.sql.functions import when, regexp_extract, col
from pyspark import sql
from pyspark.sql.functions import *

from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import VectorIndexer

import matplotlib.pyplot as plt
import sys

import seaborn as sns

pd.set_option('display.max_columns', 500)

In [ ]:
# Get Data
#Added Data types for memory optimization and faster loading
data = spark.read.csv('/content/drive/MyDrive/used_cars_data.csv', header=True, inferSchema=True).limit(50000)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.8/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
data.count()

In [ ]:
from pyspark.sql.types import IntegerType,BooleanType,DateType
import pyspark.sql.functions as F

def fromBooleanToInt(s):
    """
    This is just a simple python function to move boolean to integers.
    
    >>> fromBooleanToInt(None)
    >>> fromBooleanToInt(True)
    1
    >>> fromBooleanToInt(False)
    1
    """
    if s == True:
        return 1
    elif s==False:
        return 0
    else:
        return None 

fromBooleanToInt_udf = F.udf(lambda x: fromBooleanToInt(x), IntegerType()) 

data = data.withColumn('fleet',fromBooleanToInt_udf(data['fleet'])) 
data = data.withColumn('frame_damaged',fromBooleanToInt_udf(data['frame_damaged'])) 
data = data.withColumn('franchise_dealer',fromBooleanToInt_udf(data['franchise_dealer'])) 
data = data.withColumn('has_accidents',fromBooleanToInt_udf(data['has_accidents'])) 
data = data.withColumn('isCab',fromBooleanToInt_udf(data['isCab'])) 
data = data.withColumn('is_cpo',fromBooleanToInt_udf(data['is_cpo'])) 
data = data.withColumn('is_new',fromBooleanToInt_udf(data['is_new'])) 
data = data.withColumn('is_oemcpo',fromBooleanToInt_udf(data['is_oemcpo'])) 
data = data.withColumn('salvage',fromBooleanToInt_udf(data['salvage'])) 
data = data.withColumn('theft_title',fromBooleanToInt_udf(data['theft_title'])) 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = data.dropDuplicates()
print("Distinct count: "+str(data.count()))

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.8/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
data = data.drop("listed_date")

In [ ]:
# df = data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).toPandas().head()
amount_missing_df = data.select([(count(when(isnan(c) | col(c).isNull(), c))*100/count(lit(1))).alias(c) for c in data.columns])
amount_missing_df.show()

+---+---+------------+------+----------+----------+---------+------+----+-----------------+--------------------+------------+----------+----------------+-------------------+-----------+--------------+------+-------------+----------------+--------------+-------------+----------------+---------+-------------+------+--------------------+----------+--------------+------+------------+------+------+---------+--------+------+-------------+----------+---------+----------------+-------------+---------+---------------+-------+----------+-----------+------+-----+-------+--------------+-------------+-----+-------+-----------+------+------------+--------------------+------+---------+-----------------------+------------+--------------------+---------+-----+----+
|_c0|vin|back_legroom|   bed|bed_height|bed_length|body_type| cabin|city|city_fuel_economy|combine_fuel_economy|daysonmarket|dealer_zip|engine_cylinders|engine_displacement|engine_type|exterior_color| fleet|frame_damaged|franchise_dealer|fra

In [ ]:
data.summary().toPandas()

,summary,_c0,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,daysonmarket,dealer_zip,engine_cylinders,engine_displacement,engine_type,exterior_color,fleet,frame_damaged,franchise_dealer,franchise_make,front_legroom,fuel_tank_volume,fuel_type,has_accidents,height,highway_fuel_economy,horsepower,interior_color,isCab,is_certified,is_cpo,is_new,is_oemcpo,latitude,length,listing_color,listing_id,longitude,main_picture_url,major_options,make_name,maximum_seating,mileage,model_name,owner_count,power,price,salvage,savings_amount,seller_rating,sp_id,sp_name,theft_title,torque,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,count,1500000,1500000,1426707,9284,201277,201277,1493314,28245,1500000,1254750,0,1500000,1500000,1455242,1420546,1455242,1499996,802507,802507,1500000,1212327,1426707,1426707,1461382,802507,1426707,1254750,1420546,1499895,802507,0,100987,1500000,74704,1500000,1426707,1500000,1500000,1500000,1313002,1403537,1500000,1426707,1427012,1500000,755623,1269983,1500000,802507,1500000,1482762,1499911,1500000,802507,1252794,1472973,1472973,1445368,1444974,0,1430436,1430436,1426707,1426707,1500000
1,mean,749999.5,Infinity,None,None,None,None,None,None,None,22.365110978282527,None,76.40229266666667,29977.329819954914,None,2927.714554826102,None,5132.078242229368,0.18111742327481256,0.009238548698017587,0.8096326666666667,None,None,None,None,0.1681081909565898,None,29.17556724447101,245.496782927128,6876.763636363636,0.15378308226594908,None,1.0,0.48086,1.0,38.32248923139961,None,None,2.75414584440984E8,-80.75007164259848,None,None,None,None,31551.69577200472,1549.3704494125934,1.5207080779700988,None,29474.935587153337,0.005259767204522827,549.2656406666666,4.272535810139143,233948.8255536495,None,0.002924585081500847,None,None,None,None,131.22539267015708,None,None,None,None,None,2017.631246
2,stddev,433012.8462297622,NaN,None,None,None,None,None,None,None,7.759637445251214,None,109.32362453306646,17238.34746019152,None,1309.5414969040357,None,20381.158701200682,0.3851156801203441,0.0956724062713005,0.39259102700481857,None,None,None,None,0.3739625667814825,None,7.071351875652732,88.62671162677182,31404.24037926155,0.3607409153867586,None,0.0,0.4996336926414701,0.0,4.683470199082295,None,None,8924250.127632566,5.946522447192903,None,None,None,None,45268.289282703445,723.8195469859019,0.9043390844163363,None,18869.85359565794,0.07233331579004001,1068.0758383671564,0.5059160374603836,131033.00770750956,None,0.05400032886251098,None,None,None,None,566.4438268480721,None,None,None,None,None,4.267554017876733
3,min,0,0,--,Long,--,--,Convertible,Crew Cab,Abbeville,7.0,None,0,01060,H4,700.0,H4,"""""",0,0,0,Acura,--,--,Biodiesel,0,--,10.0,65.0,'4wd Supercrew 145'' Xlt',0,None,1,0,1,18.3467,--,BLACK,20498116,-157.928,https://058cfc5fa3837fde14de-43d0ef0c16a1c26e4...,"['101A Mid Equipment Group', '301A Mid Equipme...",AM General,--,0.0,1 Series,1.0,"1,001 hp @ 6,000 RPM",165.0,0,0,1.0,41593.0,\t Dijie Auto Sales and Service Co.,0,"100 lb-ft @ 3,000 RPM",A,1-Speed Automatic,t10000,!,None,4WD,4X2,--,--,1915
4,25%,374988,1.2870719E7,None,None,None,None,None,None,None,18.0,None,14,17042.0,None,2000.0,None,70.0,0,0,1,None,None,None,None,0,None,25.0,175.0,20.0,0,None,1,0,1,35.1692,None,None,274576666,-84.2834,None,None,None,None,6.0,1500.0,1.0,None,18000.0,0,0,4.0,63915.0,None,0,None,None,None,None,2.5,None,None,None,None,None,2017
5,50%,749911,5.262659E8,None,None,None,None,None,None,None,21.0,None,35,29708.0,None,2500.0,None,218.0,0,0,1,None,None,None,None,0,None,28.0,241.0,57.0,0,None,1,0,1,40.1209,None,None,278525093,-81.0239,None,None,None,None,10258.0,1500.0,1.0,None,26430.0,0,0,4.333333333333333,281863.0,None,0,None,None,None,None,2.5,None,None,None,None,None,2019
6,75%,1124951,9.116310298E9,None,None,None,None,None,None,None,26.0,None,82,44039.0,None,3600.0,None,300.0,0,0,1,None,None,None,None,0,None

In [ ]:
data = data.drop("bed","bed_height","bed_length","cabin","combine_fuel_economy","is_certified","is_cpo","is_oemcpo","salvage","theft_title","vehicle_damage_category","engine_cylinders", "owner_count","fleet", "isCab", "frame_damaged", "main_picture_url")

In [ ]:
data.groupBy('back_legroom').count().orderBy('count', ascending=False).show()

+------------+-----+
|back_legroom|count|
+------------+-----+
|        null|73293|
|     38.3 in|59125|
|     43.6 in|47178|
|          --|44649|
|     35.7 in|40530|
|     37.4 in|38575|
|       39 in|37315|
|     40.4 in|36412|
|     38.6 in|35435|
|     39.9 in|35350|
|       38 in|33575|
|     35.2 in|29310|
|     37.9 in|27246|
|     38.1 in|26735|
|     39.7 in|26665|
|     39.6 in|26534|
|     38.4 in|25298|
|     40.9 in|24852|
|     36.5 in|24736|
|     36.1 in|24493|
+------------+-----+
only showing top 20 rows



In [ ]:
data = data.withColumn("back_legroom",split(data["back_legroom"]," ").getItem(0))
data.withColumn("back_legroom",col("back_legroom").cast("integer"))
df_mean = data.select(mean(col('back_legroom')).alias('avg')).collect()
avg = df_mean[0]['avg']
data = data.withColumn('back_legroom', when(data.back_legroom.isNull(), lit(avg)).otherwise(data.back_legroom))
print(data.filter(col("back_legroom").isNull()).count())

0


In [ ]:
data.groupBy('body_type').count().orderBy('count', ascending=False).show()

+---------------+------+
|      body_type| count|
+---------------+------+
|SUV / Crossover|739196|
|          Sedan|360121|
|   Pickup Truck|220497|
|      Hatchback| 40944|
|        Minivan| 40165|
|          Coupe| 33009|
|            Van| 26734|
|          Wagon| 19761|
|    Convertible| 12887|
|           null|  6686|
+---------------+------+



In [ ]:
data = data.withColumn('body_type', when(data.body_type.isNull(), lit('Unknown')).otherwise(data.body_type))
print(data.filter(col("body_type").isNull()).count())

0


In [ ]:
data.groupBy('city_fuel_economy').count().orderBy('count', ascending=False).show()

+-----------------+------+
|city_fuel_economy| count|
+-----------------+------+
|             null|245250|
|             18.0|103319|
|             19.0| 98075|
|             21.0| 92727|
|             22.0| 88825|
|             20.0| 84518|
|             16.0| 84431|
|             17.0| 79379|
|             26.0| 77640|
|             25.0| 70227|
|             27.0| 69338|
|             15.0| 69110|
|             23.0| 59110|
|             24.0| 56219|
|             28.0| 47750|
|             30.0| 36804|
|             29.0| 30286|
|             14.0| 22041|
|             31.0| 16512|
|             13.0| 12679|
+-----------------+------+
only showing top 20 rows



In [ ]:
df_mean = data.select(mean(col('city_fuel_economy')).alias('avg')).collect()
avg = df_mean[0]['avg']
data = data.withColumn('city_fuel_economy', when(data.city_fuel_economy.isNull(), lit(avg)).otherwise(data.city_fuel_economy))
print(data.filter(col("city_fuel_economy").isNull()).count())

0


In [ ]:
# This is better represented in the horsepower column, we can drop this column.
data = data.drop("engine_displacement")

In [ ]:
data.groupBy('engine_type').count().orderBy('count', ascending=False).show()

+--------------------+------+
|         engine_type| count|
+--------------------+------+
|                  I4|724887|
|                  V6|371361|
|                  V8|132054|
|                null| 44758|
|                  H4| 39230|
|V8 Flex Fuel Vehicle| 37175|
|V6 Flex Fuel Vehicle| 34600|
|           I4 Hybrid| 31578|
|                  I3| 28909|
|                  I6| 13950|
|           I6 Diesel|  8692|
|        V8 Biodiesel|  8589|
|I4 Flex Fuel Vehicle|  4574|
|           I4 Diesel|  3506|
|           V6 Diesel|  3198|
|                  I5|  3018|
|           V8 Diesel|  2692|
|                  H6|  2281|
|        V6 Biodiesel|  1533|
|           V6 Hybrid|  1397|
+--------------------+------+
only showing top 20 rows



In [ ]:
data = data.withColumn('engine_type', when(data.body_type.isNull(), lit('Unknown')).otherwise(data.engine_type))
#print(data.filter(col("engine_type").isNull()).count())

In [ ]:
data.groupBy('exterior_color').count().orderBy('count', ascending=False).show()

+--------------------+-----+
|      exterior_color|count|
+--------------------+-----+
|               Black|84883|
|               White|59331|
|        Summit White|46956|
|                Gray|38891|
|              Silver|37783|
|                Blue|27824|
|        Oxford White|27477|
|                None|23463|
|                 Red|21917|
|Bright White Clea...|20484|
| Silver Ice Metallic|19153|
|        Gun Metallic|15237|
|Mosaic Black Meta...|14939|
|Diamond Black Cry...|14645|
|   Magnetic Metallic|13031|
|Satin Steel Metallic|13004|
|               BLACK|12897|
| Crystal Black Pearl|12718|
|Granite Crystal M...|12394|
|Modern Steel Meta...|10985|
+--------------------+-----+
only showing top 20 rows



In [ ]:
data = data.withColumn('exterior_color', when(data.body_type.isNull(), lit('Black')).otherwise(data.exterior_color))
#print(data.filter(col("exterior_color").isNull()).count())

In [ ]:
# Recategorize the exterior_color

In [ ]:
# The information in this column closely matches the information in the make_name column. So we can drop this column from the dataset due to redundancy.
data = data.drop("franchise_make")

In [ ]:
data.groupBy('front_legroom').count().orderBy('count', ascending=False).show()

+-------------+------+
|front_legroom| count|
+-------------+------+
|      40.9 in|116477|
|      41.3 in| 76096|
|        41 in| 73540|
|         null| 73293|
|      43.9 in| 70572|
|      41.2 in| 67424|
|      42.3 in| 57873|
|      41.5 in| 50396|
|        43 in| 47323|
|      40.8 in| 45924|
|      44.5 in| 45268|
|      42.2 in| 43806|
|      41.1 in| 41269|
|      40.3 in| 39805|
|        42 in| 39297|
|      42.4 in| 38882|
|      41.8 in| 34107|
|      43.1 in| 34026|
|      41.4 in| 32238|
|      45.3 in| 31493|
+-------------+------+
only showing top 20 rows



In [ ]:
data = data.withColumn("front_legroom",split(data["front_legroom"]," ").getItem(0))
data.withColumn("front_legroom",col("front_legroom").cast("float"))
df_mean = data.select(mean(col('front_legroom')).alias('avg')).collect()
avg = df_mean[0]['avg']
data = data.withColumn('front_legroom', when(data.front_legroom.isNull(), lit(avg)).otherwise(data.front_legroom))
#print(data.filter(col("front_legroom").isNull()).count())

In [ ]:
# Combining front_legroom and back_legroom
data = data.withColumn('legroom', expr("front_legroom + back_legroom"))

In [ ]:
data.groupBy('legroom').count().orderBy('count', ascending=False).show()


In [ ]:
data = data.drop("front_legroom", "back_legroom")

In [ ]:
data.groupBy('fuel_tank_volume').count().orderBy('count', ascending=False).show()

In [ ]:
data = data.replace({'--':None}, subset=['fuel_tank_volume'])
data = data.withColumn("fuel_tank_volume",split(data["fuel_tank_volume"]," ").getItem(0))
data.withColumn("fuel_tank_volume",col("fuel_tank_volume").cast("float"))



DataFrame[_c0: int, vin: string, body_type: string, city: string, city_fuel_economy: double, daysonmarket: int, dealer_zip: string, engine_type: string, exterior_color: string, franchise_dealer: int, fuel_tank_volume: float, fuel_type: string, has_accidents: int, height: string, highway_fuel_economy: double, horsepower: double, interior_color: string, is_new: int, latitude: double, length: string, listing_color: string, listing_id: int, longitude: double, major_options: string, make_name: string, maximum_seating: string, mileage: double, model_name: string, power: string, price: double, savings_amount: int, seller_rating: double, sp_id: double, sp_name: string, torque: string, transmission: string, transmission_display: string, trimId: string, trim_name: string, wheel_system: string, wheel_system_display: string, wheelbase: string, width: string, year: int, legroom: double]

In [ ]:
data.groupBy('fuel_tank_volume').count().orderBy('count', ascending=False).show()

In [ ]:
# Replace nulls with 0 when fuel_type = "Electric" and rest with mean

In [ ]:
data.groupBy('fuel_type').count().orderBy('count', ascending=False).show()

In [ ]:
data = data.withColumn('fuel_type', when(data.fuel_type.isNull(), lit('Gasoline')).otherwise(data.fuel_type))
#print(data.filter(col("fuel_type").isNull()).count())

In [ ]:
data.groupBy('has_accidents').count().orderBy('count', ascending=False).show()

In [ ]:
data = data.withColumn('has_accidents', when(data.has_accidents.isNull(), lit(0)).otherwise(data.has_accidents))
#print(data.filter(col("has_accidents").isNull()).count())

In [ ]:
data.groupBy('height').count().orderBy('count', ascending=False).show()

In [ ]:
data = data.replace({'--':None}, subset=['height'])
data = data.withColumn("height",split(data["height"]," ").getItem(0))
data.withColumn("height",col("height").cast("float"))

DataFrame[_c0: int, vin: string, body_type: string, city: string, city_fuel_economy: double, daysonmarket: int, dealer_zip: string, engine_type: string, exterior_color: string, franchise_dealer: int, fuel_tank_volume: string, fuel_type: string, has_accidents: int, height: float, highway_fuel_economy: double, horsepower: double, interior_color: string, is_new: int, latitude: double, length: string, listing_color: string, listing_id: int, longitude: double, major_options: string, make_name: string, maximum_seating: string, mileage: double, model_name: string, power: string, price: double, savings_amount: int, seller_rating: double, sp_id: double, sp_name: string, torque: string, transmission: string, transmission_display: string, trimId: string, trim_name: string, wheel_system: string, wheel_system_display: string, wheelbase: string, width: string, year: int, legroom: double]

In [ ]:
df_mean = data.select(mean(col('height')).alias('avg')).collect()
avg = df_mean[0]['avg']
data = data.withColumn('height', when(data.height.isNull(), lit(avg)).otherwise(data.height))
#print(data.filter(col("height").isNull()).count())

In [ ]:
df_mean = data.select(mean(col('highway_fuel_economy')).alias('avg')).collect()
avg = df_mean[0]['avg']
data = data.withColumn('highway_fuel_economy', when(data.highway_fuel_economy.isNull(), lit(avg)).otherwise(data.highway_fuel_economy))
#print(data.filter(col("highway_fuel_economy").isNull()).count())

In [ ]:
data.groupBy('horsepower').count().orderBy('count', ascending=False).show()

In [ ]:
data = data.withColumn('horsepower', when(data.horsepower.isNull(), lit(170)).otherwise(data.horsepower))
#print(data.filter(col("horsepower").isNull()).count())

In [ ]:
data.groupBy('interior_color').count().orderBy('count', ascending=False).show()

In [ ]:
data = data.withColumn('interior_color', when(data.interior_color.isNull(), lit("Other")).otherwise(data.interior_color))
#print(data.filter(col("interior_color").isNull()).count())

In [ ]:
data.groupBy('length').count().orderBy('count', ascending=False).show()

In [ ]:
data = data.replace({'--':None}, subset=['length'])
data = data.withColumn("length",split(data["length"]," ").getItem(0))
data.withColumn("length",col("length").cast("float"))
df_mean = data.select(mean(col('length')).alias('avg')).collect()
avg = df_mean[0]['avg']
data = data.withColumn('length', when(data.length.isNull(), lit(avg)).otherwise(data.length))
#print(data.filter(col("length").isNull()).count())

In [ ]:
data = data.replace({'--':None}, subset=['wheelbase'])
data = data.withColumn("wheelbase",split(data["wheelbase"]," ").getItem(0))
data.withColumn("wheelbase",col("wheelbase").cast("float"))
df_mean = data.select(mean(col('wheelbase')).alias('avg')).collect()
avg = df_mean[0]['avg']
data = data.withColumn('wheelbase', when(data.wheelbase.isNull(), 
                                         lit(avg)).otherwise(data.wheelbase))
#print(data.filter(col("wheelbase").isNull()).count())

In [ ]:
data = data.replace({'--':None}, subset=['width'])
data = data.withColumn("width",split(data["width"]," ").getItem(0))
data.withColumn("width",col("width").cast("float"))
df_mean = data.select(mean(col('width')).alias('avg')).collect()
avg = df_mean[0]['avg']
data = data.withColumn('width', when(data.width.isNull(), lit(avg)).otherwise(data.width))
#print(data.filter(col("width").isNull()).count())

In [ ]:
data.groupBy('major_options').count().orderBy('count', ascending=False).show()

In [ ]:
data = data.withColumn("major_options_count",size(split(col("major_options"),",")))
#data.groupBy('major_options_count').count().orderBy('count', ascending=False).show()

In [ ]:
df_mean = data.select(mean(col('major_options_count')).alias('avg')).collect()
avg = df_mean[0]['avg']
data = data.withColumn('major_options_count', when(data.major_options_count.isNull(), lit(avg)).otherwise(data.major_options_count))
data = data.drop("major_options")
#print(data.filter(col("major_options_count").isNull()).count())

In [ ]:
data.groupBy('maximum_seating').count().orderBy('count', ascending=False).show()

In [ ]:
data = data.replace({'--':None}, subset=['maximum_seating'])
data = data.withColumn("maximum_seating",split(data["maximum_seating"]," ").getItem(0))
data.withColumn("maximum_seating",col("maximum_seating").cast("int"))
df_mean = data.select(mean(col('maximum_seating')).alias('avg')).collect()
avg = df_mean[0]['avg']
data = data.withColumn('maximum_seating', when(data.maximum_seating.isNull(), lit(avg)).otherwise(data.maximum_seating))
#print(data.filter(col("maximum_seating").isNull()).count())

In [ ]:
amount_missing_df = data.select([(count(when(isnan(c) | col(c).isNull(), c))*100/count(lit(1))).alias(c) for c in data.columns])
amount_missing_df.show()

# Model

In [ ]:
data.show()

+-------+-----------------+---------------+----------+------------------+------------+----------+-----------+--------------------+----------------+----------------+---------+-------------+-----------------+--------------------+----------+--------------------+------+--------+------------------+-------------+----------+---------+---------+-----------------+-------+--------------------+-----+-------+--------------+------------------+--------+--------------------+------+------------+--------------------+------+-----------+------------+--------------------+------------------+-----------------+----+-----------------+-------------------+
|    _c0|              vin|      body_type|      city| city_fuel_economy|daysonmarket|dealer_zip|engine_type|      exterior_color|franchise_dealer|fuel_tank_volume|fuel_type|has_accidents|           height|highway_fuel_economy|horsepower|      interior_color|is_new|latitude|            length|listing_color|listing_id|longitude|make_name|  maximum_seating|mile

In [ ]:
data = data.drop('_c0', 'vin', 'city','dealer_zip', 'exterior_color', 'interior_color', 'listing_id', 'make_name','model_name', 'power', 'sp_name', 'torque',	'trimId',	'trim_name',	'wheel_system_display', 'sp_id')

In [ ]:
dummy = data.toPandas()


In [ ]:
dummy_cols = ['body_type','engine_type', 'fuel_type', 'listing_color', 'transmission', 'transmission_display', 'wheel_system']
dummy_df = pd.get_dummies(dummy, columns=dummy_cols)
dummy_df.head()

,city_fuel_economy,daysonmarket,franchise_dealer,fuel_tank_volume,has_accidents,height,highway_fuel_economy,horsepower,is_new,latitude,length,longitude,maximum_seating,mileage,price,savings_amount,seller_rating,wheelbase,width,year,legroom,major_options_count,body_type_Convertible,body_type_Coupe,body_type_Hatchback,body_type_Minivan,body_type_Pickup Truck,body_type_SUV / Crossover,body_type_Sedan,body_type_Unknown,body_type_Van,body_type_Wagon,engine_type_H4,engine_type_H4 Hybrid,engine_type_H6,engine_type_I2,engine_type_I3,engine_type_I3 Hybrid,engine_type_I4,engine_type_I4 Compressed Natural Gas,engine_type_I4 Diesel,engine_type_I4 Flex Fuel Vehicle,engine_type_I4 Hybrid,engine_type_I5,engine_type_I5 Biodiesel,engine_type_I5 Diesel,engine_type_I6,engine_type_I6 Diesel,engine_type_I6 Hybrid,engine_type_R2,engine_type_V10,engine_type_V10 Diesel,engine_type_V12,engine_type_V6,engine_type_V6 Biodiesel,engine_type_V6 Compressed Natural Gas,engine_type_V6 Diesel,engine_type_V6 Flex Fuel Vehicle,engine_type_V6 Hybrid,engine_type_V8,engine_type_V8 Biodiesel,engine_type_V8 Compressed Natural Gas,engine_type_V8 Diesel,engine_type_V8 Flex Fuel Vehicle,engine_type_V8 Hybrid,engine_type_V8 Propane,engine_type_W12,engine_type_W12 Flex Fuel Vehicle,engine_type_W16,engine_type_W8,fuel_type_Biodiesel,fuel_type_Compressed Natural Gas,fuel_type_Diesel,fuel_type_Electric,fuel_type_Flex Fuel Vehicle,fuel_type_Gasoline,fuel_type_Hybrid,fuel_type_Propane,listing_color_BLACK,listing_color_BLUE,listing_color_BROWN,listing_color_GOLD,listing_color_GRAY,listing_color_GREEN,listing_color_ORANGE,listing_color_PINK,listing_color_PURPLE,listing_color_RED,listing_color_SILVER,listing_color_TEAL,listing_color_UNKNOWN,listing_color_WHITE,listing_color_YELLOW,transmission_A,transmission_CVT,transmission_Dual Clutch,transmission_M,transmission_display_1-Speed Automatic,transmission_display_1-Speed CVT,transmission_display_1-Speed Dual Clutch,transmission_display_10-Speed Automatic,transmission_display_10-Speed CVT,transmission_display_2-Speed Automatic,transmission_display_3-Speed Automatic,transmission_display_3-Speed Automatic Overdrive,transmission_display_3-Speed Manual,transmission_display_4-Speed Automatic,transmission_display_4-Speed Automatic Overdrive,transmission_display_4-Speed CVT,transmission_display_4-Speed Dual Clutch,transmission_display_4-Speed Manual,transmission_display_4-Speed Manual Overdrive,transmission_display_5-Speed Automatic,transmission_display_5-Speed Automatic Overdrive,transmission_display_5-Speed Dual Clutch,transmission_display_5-Speed Manual,transmission_display_5-Speed Manual Overdrive,transmission_display_6-Speed Automatic,transmission_display_6-Speed Automatic Overdrive,transmission_display_6-Speed CVT,transmission_display_6-Speed Dual Clutch,transmission_display_6-Speed Manual,transmission_display_6-Speed Manual Overdrive,transmission_display_7-Speed Automatic,transmission_display_7-Speed Automatic Overdrive,transmission_display_7-Speed CVT,transmission_display_7-Speed Dual Clutch,transmission_display_7-Speed Manual,transmission_display_7-Speed Manual Overdrive,transmission_display_8-Speed Automatic,transmission_display_8-Speed Automatic Overdrive,transmission_display_8-Speed CVT,transmission_display_8-Speed Dual Clutch,transmission_display_8-Speed Manual,transmission_display_9-Speed Automatic,transmission_display_9-Speed Automatic Overdrive,transmission_display_Automatic,transmission_display_Continuously Variable Transmission,transmission_display_Manual,wheel_system_4WD,wheel_system_4X2,wheel_system_AWD,wheel_system_FWD,wheel_system_RWD
0,22.365111,105,0,None,1,65.88426874843415,29.175567,170.0,0,42.0994,192.95317048549902,-70.9612,5.454915665353393,79499.0,36995.0,0,3.000000,114.79503438858377,78.30550440387708,2016,80.098705,1.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [ ]:
#drop columns
#vin, dealer_zip, exterior_color, interior_color, listing_id, model_name, power, sp_name, torque	trimId	trim_name	wheel_system_display, 
dummy_df = dummy_df.drop(['_c0', 'vin', 'city','dealer_zip', 'exterior_color', 'interior_color', 'listing_id', 'make_name','model_name', 'power', 'sp_name', 'torque',	'trimId',	'trim_name',	'wheel_system_display', 'sp_id' ], axis = 1)
dummy_df.head()

In [ ]:
data_reg = spark.createDataFrame(dummy_df)


In [ ]:
data_reg.columns

['city_fuel_economy',
 'daysonmarket',
 'franchise_dealer',
 'fuel_tank_volume',
 'has_accidents',
 'height',
 'highway_fuel_economy',
 'horsepower',
 'is_new',
 'latitude',
 'length',
 'longitude',
 'maximum_seating',
 'mileage',
 'price',
 'savings_amount',
 'seller_rating',
 'wheelbase',
 'width',
 'year',
 'legroom',
 'major_options_count',
 'body_type_Convertible',
 'body_type_Coupe',
 'body_type_Hatchback',
 'body_type_Minivan',
 'body_type_Pickup Truck',
 'body_type_SUV / Crossover',
 'body_type_Sedan',
 'body_type_Unknown',
 'body_type_Van',
 'body_type_Wagon',
 'engine_type_H4',
 'engine_type_H4 Hybrid',
 'engine_type_H6',
 'engine_type_I2',
 'engine_type_I3',
 'engine_type_I3 Hybrid',
 'engine_type_I4',
 'engine_type_I4 Compressed Natural Gas',
 'engine_type_I4 Diesel',
 'engine_type_I4 Flex Fuel Vehicle',
 'engine_type_I4 Hybrid',
 'engine_type_I5',
 'engine_type_I5 Biodiesel',
 'engine_type_I5 Diesel',
 'engine_type_I6',
 'engine_type_I6 Diesel',
 'engine_type_I6 Hybrid',
 

In [ ]:
columns_list = ['city_fuel_economy',
 'daysonmarket',
 'franchise_dealer',
 'fuel_tank_volume',
 'has_accidents',
 'height',
 'highway_fuel_economy',
 'horsepower',
 'is_new',
 'latitude',
 'length',
 'longitude',
 'maximum_seating',
 'mileage',
 'price',
 'savings_amount',
 'seller_rating',
 'wheelbase',
 'width',
 'year',
 'legroom',
 'major_options_count']
for col in columns_list:
  data_reg = data_reg.withColumn(col, data_reg[col].cast('Integer'))
     

In [ ]:
data_reg = data_reg.na.fill(0)

In [ ]:
data_reg.show(20)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.8/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
input_cols = ['city_fuel_economy',
 'daysonmarket',
 'franchise_dealer',
 'fuel_tank_volume',
 'has_accidents',
 'height',
 'highway_fuel_economy',
 'horsepower',
 'is_new',
 'latitude',
 'length',
 'longitude',
 'maximum_seating',
 'mileage',
 'savings_amount',
 'seller_rating',
 'wheelbase',
 'width',
 'year',
 'legroom',
 'major_options_count',
 'body_type_Convertible',
 'body_type_Coupe',
 'body_type_Hatchback',
 'body_type_Minivan',
 'body_type_Pickup Truck',
 'body_type_SUV / Crossover',
 'body_type_Sedan',
 'body_type_Unknown',
 'body_type_Van',
 'body_type_Wagon',
 'engine_type_H4',
 'engine_type_I3',
 'engine_type_I4',
 'engine_type_I4 Diesel',
 'engine_type_I4 Flex Fuel Vehicle',
 'engine_type_I4 Hybrid',
 'engine_type_I5',
 'engine_type_I6',
 'engine_type_I6 Diesel',
 'engine_type_R2',
 'engine_type_V10',
 'engine_type_V12',
 'engine_type_V6',
 'engine_type_V6 Diesel',
 'engine_type_V6 Flex Fuel Vehicle',
 'engine_type_V6 Hybrid',
 'engine_type_V8',
 'engine_type_V8 Biodiesel',
 'engine_type_V8 Diesel',
 'engine_type_V8 Flex Fuel Vehicle',
 'engine_type_W12',
 'fuel_type_Biodiesel',
 'fuel_type_Diesel',
 'fuel_type_Electric',
 'fuel_type_Flex Fuel Vehicle',
 'fuel_type_Gasoline',
 'fuel_type_Hybrid',
 'listing_color_BLACK',
 'listing_color_BLUE',
 'listing_color_BROWN',
 'listing_color_GOLD',
 'listing_color_GRAY',
 'listing_color_GREEN',
 'listing_color_ORANGE',
 'listing_color_PURPLE',
 'listing_color_RED',
 'listing_color_SILVER',
 'listing_color_TEAL',
 'listing_color_UNKNOWN',
 'listing_color_WHITE',
 'listing_color_YELLOW',
 'transmission_A',
 'transmission_CVT',
 'transmission_Dual Clutch',
 'transmission_M',
 'transmission_display_1-Speed Automatic',
 'transmission_display_2-Speed Automatic',
 'transmission_display_3-Speed Automatic',
 'transmission_display_4-Speed Automatic',
 'transmission_display_4-Speed Automatic Overdrive',
 'transmission_display_5-Speed Automatic',
 'transmission_display_5-Speed Automatic Overdrive',
 'transmission_display_5-Speed Manual',
 'transmission_display_6-Speed Automatic',
 'transmission_display_6-Speed Automatic Overdrive',
 'transmission_display_6-Speed Dual Clutch',
 'transmission_display_6-Speed Manual',
 'transmission_display_6-Speed Manual Overdrive',
 'transmission_display_7-Speed Automatic',
 'transmission_display_7-Speed CVT',
 'transmission_display_7-Speed Dual Clutch',
 'transmission_display_8-Speed Automatic',
 'transmission_display_8-Speed Automatic Overdrive',
 'transmission_display_9-Speed Automatic',
 'transmission_display_9-Speed Automatic Overdrive',
 'transmission_display_Automatic',
 'transmission_display_Continuously Variable Transmission',
 'transmission_display_Manual',
 'wheel_system_4WD',
 'wheel_system_4X2',
 'wheel_system_AWD',
 'wheel_system_FWD',
 'wheel_system_RWD']
vectorAssembler = VectorAssembler(inputCols = input_cols, outputCol = 'features')
vdata_reg = vectorAssembler.transform(data_reg)
vdata_reg = vdata_reg.select(['features', 'price'])
#vdata_reg.show(10)

In [ ]:
#Splitting the dataset 
splits = vdata_reg.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]     

In [ ]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='price', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

In [ ]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 11798.249436
r2: 0.690410


In [ ]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","price","features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="price",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+------------------+-----+--------------------+
|        prediction|price|            features|
+------------------+-----+--------------------+
|118053.74002655968|82980|(149,[0,1,2,3,4,5...|
| 23797.74296258227|17600|(149,[0,1,2,3,4,5...|
| 24462.97811441496|10969|(149,[0,1,2,3,4,5...|
|38109.741744517814|23500|(149,[0,1,2,3,4,5...|
| 38129.89629907068|28778|(149,[0,1,2,3,4,5...|
+------------------+-----+--------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.648903


In [ ]:
test_result = lr_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 13458.2


In [ ]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","price","features").show()

+------------------+-----+--------------------+
|        prediction|price|            features|
+------------------+-----+--------------------+
|118053.74002655968|82980|(149,[0,1,2,3,4,5...|
| 23797.74296258227|17600|(149,[0,1,2,3,4,5...|
| 24462.97811441496|10969|(149,[0,1,2,3,4,5...|
|38109.741744517814|23500|(149,[0,1,2,3,4,5...|
| 38129.89629907068|28778|(149,[0,1,2,3,4,5...|
|42100.182450144785|30500|(149,[0,1,2,3,4,5...|
| 2866.174145102501| 5800|(149,[0,1,2,3,4,5...|
|12743.750435486203| 6888|(149,[0,1,2,3,4,5...|
|16021.165657255799|16432|(149,[0,1,2,3,4,5...|
|14650.719623595709|17980|(149,[0,1,2,3,4,5...|
| 14499.15415922366|17926|(149,[0,1,2,3,4,5...|
|17155.402737981174|22991|(149,[0,1,2,3,4,5...|
| 25805.63406400825|28711|(149,[0,1,2,3,4,5...|
| 17402.47273917636|13990|(149,[0,1,2,3,4,5...|
|25010.154622446746|19995|(149,[0,1,2,3,4,5...|
|16347.005683500087|15480|(149,[0,1,2,3,4,5...|
| 26129.29545766604|20369|(149,[0,1,2,3,4,5...|
|23879.640407619067|22495|(149,[0,1,2,3,

In [ ]:
# Decision tree

In [ ]:
from pyspark.ml.regression import DecisionTreeRegressor
dt = DecisionTreeRegressor(featuresCol ='features', labelCol = 'price')
dt_model = dt.fit(train_df)
dt_predictions = dt_model.transform(test_df)
dt_evaluator = RegressionEvaluator(
    labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = dt_evaluator.evaluate(dt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 15191.3


In [ ]:
dt_model.featureImportances

SparseVector(149, {3: 0.0332, 7: 0.3754, 13: 0.3123, 18: 0.0187, 19: 0.0253, 20: 0.0159, 26: 0.0049, 38: 0.0108, 42: 0.0441, 83: 0.1559, 147: 0.0034})

In [ ]:
d = data_reg.take(1)


In [ ]:
a = data_reg.toPandas()

In [ ]:
a.iloc[:,82]

0        0
1        0
2        0
3        0
4        0
        ..
49995    0
49996    0
49997    0
49998    0
49999    0
Name: make_name_Alfa Romeo, Length: 50000, dtype: int64

In [ ]:
# gboost

In [ ]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol = 'features', labelCol = 'price', maxIter=10)
gbt_model = gbt.fit(train_df)
gbt_predictions = gbt_model.transform(test_df)
gbt_predictions.select('prediction', 'price', 'features').show(5)

+------------------+-----+--------------------+
|        prediction|price|            features|
+------------------+-----+--------------------+
| 72491.85175068559|82980|(149,[0,1,2,3,4,5...|
|14654.013965511542|17600|(149,[0,1,2,3,4,5...|
| 7080.732430647494|10969|(149,[0,1,2,3,4,5...|
|29691.781854025496|23500|(149,[0,1,2,3,4,5...|
|  43027.4566443404|28778|(149,[0,1,2,3,4,5...|
+------------------+-----+--------------------+
only showing top 5 rows



In [ ]:
gbt_evaluator = RegressionEvaluator(
    labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = gbt_evaluator.evaluate(gbt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 13636
